# 添加到区块链

### 区块添加原则

Simchain中的区块添加原则，按照如下两个步骤处理：
1. 定位“父区块”高度
2. 判断“父区块”的位置

如果“父区块”为末尾区块，则直接链接其后；如果“父区块”为末尾区块的前一区块（次末尾区块），则判断当前区块编号与末尾区块编号的大小，编号小的被写进区块链。如果末尾区块的编号小，则直接放弃当前区块，反之，用当前区块代替末尾区块。如果“父区块”在其他位置，直接放弃当期区块。如果找不到“父区块”，则将其存放到“孤块”列表。

In [1]:
# 根据哈希值定位区块的高度
def locate_block_by_hash(peer, block_hash):
    for height, block in enumerate(peer.blockchain):
        if block.hash == block_hash:
            return height + 1
    return None

# 尝试添加到区块链，peer：节点，block：待添加的区块
def try_to_add_block(peer, block):
    # 获取区块的前哈希值
    prev_hash = block.prev_block_hash
    # 定位“父区块”高度
    height = locate_block_by_hash(peer, prev_hash)
    # 如果找不到“父区块”，添加至“孤块”列表
    if not height:
        logger.info('{0}(pid={1} find a orphan {2})'.format(peer, peer.pid, block))
        peer.orphan_block.append(block)
        return False
    
    # 如果“父区块”在末尾
    if height == peer.get_height():
        # 将区块添加到区块链
        peer.blockchain.append(block)
        # 更新UTXO_SET和交易池
        recieve_new_prev_hash_block(peer, block.txs)
        return True

    # 如果“父区块”为末尾区块的前一个区块
    elif height == peer.get_height() - 1:
        b1, b2 = peer.blockchain[-1], block
        # 比较末尾区块与当前区块哈希值大小
        # Simchain中的区块哈希值是十六进制字符串
        a, b = int(b1.hash, 16), int(b2.hash, 16)
        # 如果末尾区块哈希值更小，保留
        if a < b:
            return False
        # 如果当前哈希值更小
        else:
            # 将末尾区块从区块链中删除
            peer.blookchian.pop()
            # 将当前区块添加到区块链中
            peer.blockchain.append(block)
            # 数据回滚并更新UTXO_SET和交易池
            recieve_exist_prev_hash_block(peer, block.txs)
    # 如果“父区块”在其他位置，返回False
    else:
        return False

In [2]:
from simchain import Network
net = Network()
zhangsan, lisi = net.peers[0], net.peers[1]
net.make_random_transactions()
zhangsan.create_candidate_block()
# 张三创建一个候选区块
block = zhangsan.candidate_block
txs = block.txs

2019-06-14 10:29:56,784 - A blockchain p2p network created,12 peers joined
2019-06-14 10:29:56,804 - genesis block has been generated
2019-06-14 10:29:56,846 - peer(50, 3)(pid=3) created a transaction
2019-06-14 10:29:56,847 - peer(50, 3)(pid=3) sent a transaction to network
2019-06-14 10:29:57,686 - peer(50, 3)(pid=3)'s transaction verified by 9 peers


In [3]:
from simchain.consensus import mine
# 计算nonce
nonce = mine(block)

In [4]:
from simchain.peer import locate_block_by_hash, try_to_add_block
# 前区块高度
locate_block_by_hash(lisi, block.prev_block_hash)

1

In [5]:
# 李四添加区块成功
try_to_add_block(lisi, block)

True

In [6]:
# 张三的交易池有3条交易
len(zhangsan.mem_pool)

1

In [7]:
# 李四的交易池为空
lisi.mem_pool

{}

In [8]:
# 李四的UTXO_SET
len(lisi.utxo_set)

14

In [9]:
# 张三的UTXO_SET
len(zhangsan.utxo_set)

14

之所以张三和李四的数据不一致，是因为李四添加了区块，对UTXO_SET和交易池进行了更新，而张三没有。

### 更新UTXO_SET和交易池

更新UTXO_SET的过程：  
1. 将区块交易所有输入单元使用过的UTXO从UTXO_SET中移除。
2. 如果系统允许从有效交易中提取UTXO，则只需要修改区块交易所有输出单元在UTXO_SET对应的UTXO的状态为确认。如果系统不允许从有效交易中提取UTXO，则需要将区块交易的所有输出单元封装为已确认的UTXO添加到UTXO_SET。  
  
更新交易池的过程：如果区块交易在节点的交易池中，则移除。

In [10]:
# 接受前区块哈希值在末尾的区块，peer：节点，txs：区块交易
def recieve_new_prev_hash_block(peer, txs):
    utxo_set, pool = peer.utxo_set, peer.mem_pool
    allow_utxo_from_pool = peer.allow_utxo_from_pool
    
    # 将交易使用过的UTXO从UTXO_SET中移除并进行备份
    peer._utxos_from_vins = remove_spent_utxo_from_txs(utxo_set, txs)
    
    # 将区块交易的所有输出单元封装成已确认的UTXO，并将其添加到UTXO_SET并进行备份
    peer._pointers_from_vouts, peer._utxos_from_vouts = confirm_utxos_from_txs(
        utxo_set, txs, allow_utxo_from_pool
    )
    
    # 将区块交易从交易池中移除，并进行备份
    peer._txs_removed = remove_txs_from_pool(pool, txs)

### 移除使用过的UTXO

In [11]:
def remove_spent_utxo_from_txs(utxo_set, txs):
    pointers = find_vin_pointer_from_txs(txs)
    utxos = delete_utxo_by_pointers(utxo_set, pointers)
    # 返回移除的UTXO列表，用于备份
    return utxos

# 找到交易所有输入单元使用过的UTXO的定位指针，并存储到列表中
def find_vin_pointer_from_txs(txs):
    return [vin.to_spend for tx in txs for vin in tx.tx_in]

# 从UTXO_SET中通过定位指针移除使用过的UTXO，并存储在列表中
def delete_utxo_by_pointers(utxo_set, pointers):
    # 初始化UTXO列表
    utxos_from_vins = []
    
    # 遍历所有定位指针
    for pointer in pointers:
        # 如果指针属于UTXO_SET
        if pointer in utxo_set:
            # 将指针对应的UTXO添加到新列表中
            utxos_from_vins.append(utxo_set[pointer])
            # 从UTXO_SET中移除该UTXO
            del utxo_set[pointer]
    
    # 返回新列表，用于备份
    return utxos_from_vins

In [12]:
from simchain.peer import find_vin_pointer_from_txs, delete_utxo_by_pointers, remove_spent_utxo_from_txs
# 复制张三的交易池
utxo_set = zhangsan.utxo_set.copy()
# 找到要删除UTXO的定位指针
pointers = find_vin_pointer_from_txs(txs)
pointers

[None,
 Pointer(tx_id:1f590447725c9071571554db5104aba498cb3e5aa09b0e5b41a36535e2382c9b,n:3)]

In [13]:
# 通过定位指针移除UTXO
utxos = delete_utxo_by_pointers(utxo_set, pointers)

In [14]:
from simchain.peer import add_utxos_to_set
# 复制移除后的UTXO_SET
utxo_set1 = utxo_set.copy()
# 还原UTXO_SET，数据回滚
add_utxos_to_set(utxo_set1, utxos)
utxo_set1 == zhangsan.utxo_set

True

In [15]:
# 删除交易使用过的UTXO
utxos1 = remove_spent_utxo_from_txs(utxo_set1, txs)
utxos1 == utxos

True

### 添加UTXO至UTXO_SET

In [16]:
# 分两种情况讨论
def confirm_utxos_from_txs(utxo_set, txs, allow_utxo_from_pool):
    
    # 如果系统允许从有效交易中提取UTXO创建交易
    if allow_utxo_from_pool:
        # 找到非创币交易中的未确认UTXO
        utxos = find_utxos_from_txs(txs[1:])
        # 将交易的所有输出单元封装成UTXO，添加至UTXO_SET，所有UTXO为已确认
        # 实际上，除了创币交易输出单元封装的UTXO，其他UTXO都已存在与UTXO_SET
        # 于是用全新的同key不同value的UTXO代替
        add_utxo_from_block_to_set(utxo_set, txs)
        
        # 找到所有UTXO的定位指针，用于备份
        pointers = find_vout_pointer_from_txs(txs)
        return pointers, utxos
    
    # 如果系统不允许
    else:
        # 找到交易中所有输出单元的UTXO，状态为已确认
        utxos = find_utxos_from_block(txs)
        # 找到所有UTXO的定位指针，用于备份
        pointers = find_vout_pointer_from_txs(txs)
        # 将UTXO添加到UTXO_SET
        add_utxos_to_set(utxo_set, utxos)
        return pointers, []

# 找到交易中的所有输出单元的定位指针，并存储到列表中 
def find_vout_pointer_from_txs(txs):
    return [Pointer(tx.id, i) for tx in txs for i, vout in enumerate(tx.tx_out)]

对于系统允许从有效交易中提取UTXO中的情况，备份的UTXO列表中不包括创币交易输出单元封装的UTXO；而系统不允许时，不需要备份UTXO，只需要备份其定位指针。原因是，在数据回滚时，会将之前添加到UTXO_SET的已确认UTXO通过定位指针删除，然后再添加备份的UTXO列表到UTXO_SET。

In [17]:
# UTXO_SET长度
len(utxo_set)

13

In [18]:
# UTXO的状态
[utxo_set[pointer].confirmed for pointer in utxo_set]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False]

In [19]:
from simchain.peer import confirm_utxos_from_txs
# 修改状态
confirm_utxos_from_txs(utxo_set, txs, True)
[utxo_set[pointer].confirmed for pointer in utxo_set]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [20]:
# 增加一个UTXO，是创币交易的输出单元封装而来
len(utxo_set)

14

由于Simchain默认允许从有效交易中提取UTXO创建交易，UTXO_SET中存在由有效交易输出单元封装的UTXO，只是在写进区块链前，状态为未确认。执行添加后，状态都修改为已确认，而且有创币交易输出单元封装的UTXO也被添加到UTXO_SET中。

In [21]:
# 从交易池中移除区块交易
def remove_txs_from_pool(pool, txs):
    # 初始化将移除的交易字典
    n_txs = {}
    # 遍历区块交易列表
    for tx in txs:
        # 如果交易在交易池中
        if tx.id in pool:
            # 则将交易添加到字典中
            n_txs[tx.id] = tx
            # 交易从交易池中移除
            del pool[tx.id]
    # 返回从交易池中移除的交易，用于备份
    return n_txs

In [22]:
from simchain.peer import remove_txs_from_pool
# 复制张三的交易池
pool = zhangsan.mem_pool.copy()
# 从pool中移除交易
remove_txs = remove_txs_from_pool(pool, txs)
pool

{}

In [23]:
# 备份的交易和张三的交易池相同
remove_txs == zhangsan.mem_pool

True

对于系统不允许从有效交易中提取UTXO创建交易的情况，需要用新区块代替末尾区块，多一个数据回滚的过程，回滚到末尾区块未加入区块链之前的状态。

In [24]:
# peer：节点，txs：区块交易
def recieve_exist_prev_hash_block(peer, txs):
    # 数据回滚
    roll_back(peer)
    # 将新区块添加到区块链
    recieve_new_prev_hash_block(peer, txs)

### 数据回滚

In [25]:
def roll_back(peer):
    # 将移除的交易重新添加到交易池中
    peer.mem_pool.update(peer._txs_removed)
    
    # 将交易使用过的UTXO重新添加到UTXO_SET
    add_utxos_to_set(peer.utxo_set, peer._utxos_from_vins)
    
    # 将交易输出单元封装的已确认UTXO从UTXO_SET中移除
    remove_utxos_from_set(peer.utxo_set, peer._pointers_from_vouts)
    
    # 如果系统允许从有效交易中提取UTXO，则将区块中非创币交易的所有输出单元封装为未确认UTXO，添加到UTXO_SET
    # 如果系统不允许，则该操作不改变数据
    add_utxos_to_set(peer.utxo_set, peer._utxos_from_vouts)
    
    # 清空节点备份数据
    peer._utxos_from_vins = []
    peer._pointers_from_vouts = []
    peer._utxos_from_vouts = []
    peer._txs_removed = {}

In [26]:
# 复制张三的UTXO_SET
utxo_set = zhangsan.utxo_set.copy()
# 复制区块交易
txs = block.txs.copy()
# 添加区块
try_to_add_block(zhangsan, block)

True

In [27]:
# 交易池变为空
zhangsan.mem_pool

{}

In [28]:
from simchain.peer import roll_back
# 数据回滚到添加区块前
roll_back(zhangsan)
# 交易回滚成功
txs[1:] == list(zhangsan.mem_pool.values())

True

In [29]:
# UTXO_SET回滚成功
zhangsan.utxo_set == utxo_set

True